## **<font color="red">Artifacts</font>**
- **Definition:** An Artifact is essentially a piece of binary data (like the content of a file) identified by a unique `filename` string within aspecific scope(session or user). Each time you save an artifact with the same filename, a new version is created.
- **Representation:** Artifacts are consistently represented using the standard `google.genai.types.Part` object. The core data is typically stored within an inline data structure of the `Part` (accessed via `inline_data`), which itself contains:
  - `data`: The raw binary content as bytes.
  - `mime_type`: A string indicating the type of the data(e.g., `"image/png"`, `"applicaiton/pdf`"). This is essential for correctly interpreting the data later.
- **Persistence & Management:** Artifacts are not stored directly within the agent or session state. Their storage and retrieval are managed by a dedicated **Artifact Service** (an implementation of `BaseArtifactService`, defined in `google.adk.artifacts`. ):
  - An in-memory service for testing or temporary storage (e.g., `InMemoryArtifactService`, defined in `google.adk.artifacts.in_memory_artifact_service.py`).
  - A service for persistent storage using GCP (e.g, `GcsArtifactService` defined in `google.adk.artifacts.gcs_artifact_service.py`).

### **Why use Artifacts**
While session `state` is suitable for storing small pieces of configuration or conversational context (like strings, numbers, booleans, or small dictionaries/lists), Artifacts are designed for scenarios involving binary or large data:
1. **Handling Non-Textual Data:** Easily store and retrieve _images_, _audio clips_, _video snippets_, _PDFs_, _spreadsheets_, or any other file format relevant to your agent's function.
2. **Persisting Large Data:** Session state is generally not optimized for storing large amounts of data. Artifacts provide a dedicated mechanism for persisting larger blobs without cluttering the session state.
3. **User File Management:** Provide capabilities for users to upload files (which can be saved as artifacts) and retrieve or download files generated by the agent (loaded from artifacts).
4. **Sharing Outputs:** Enable tools or agents to generate binary outputs (like a PDF report or a generated image) that can be saved via `save_artifact` and later accessed by other parts of the application or even in subsequent sessions (if using user namespacing).
5. **Caching Binary Data:** Store the results of computationally expensive operations that produce binary data (e.g., rendering a complex chart image) as artifacts to avoid regenerating them on subsequent requests.
In essence, whenever your agent needs to work with file-like binary data that needs to be persisted, versioned, or shared, Artifacts managed by an `ArtifactService` are the appropriate mechanism within ADK.

### **Common Use Cases**
1. **Generated Report/Files:** A tool or agent generates a report (e.g., a _PDF analysis_, a _CSV data export_, an _image chart_).
2. **Handling User Uploads:** A user uploads a file (e.g., an _image for analysis_, a _document for summarization_) through a front-end interface.
3. **Storing Intermediate Binary Results:** An agent performs a complex multi-step process where one step generates intermediate binary data (e.g., audio synthesis, simulation results).
4. **Persistent User Data:** Storing user-specific configuration or data that isn't a simple key-value state.
5. **Caching Generated Binary Content:** An agent frequently generates the same binary output based on certain inputs (e.g., a company logo image, a standard audio greeting).

In [10]:
# ============================================================
# ADK Artifact Demo (CORRECT FOR YOUR INSTALLED VERSION)
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.5-flash"
APP_NAME = "artifact_demo_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: CREATE ARTIFACT
# ============================================================

async def create_report(topic: str, tool_context=None) -> str:

    report_content = f"""
===== REPORT =====
Topic: {topic}

This is a generated report about {topic}.
{tool_context}

==================
"""

    artifact_name = f"{topic}_report.txt"

    # Wrap bytes inside types.Part
    artifact_part = types.Part.from_bytes(
        data=report_content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=artifact_name,
        artifact=artifact_part
    )

    return f"Report saved as '{artifact_name}' (version {version})"

create_report_tool = FunctionTool(func=create_report)

# ============================================================
# TOOL 2: READ ARTIFACT
# ============================================================

async def read_report(artifact_name: str, tool_context=None) -> str:

    artifact_part = await tool_context.load_artifact(
        filename=artifact_name
    )

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        return artifact_part.inline_data.data.decode("utf-8")

    return "Artifact exists but has no binary content."

read_report_tool = FunctionTool(func=read_report)

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactAgent",
    model=MODEL,
    instruction="""
You are an assistant that:
- Generates reports and stores them as artifacts.
- Reads previously stored artifacts.

Always use tools when needed.
""",
    tools=[create_report_tool, read_report_tool]
)

# ============================================================
# RUNNER
# ============================================================

async def run_chat():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("Artifact Demo Started")
    print("Try:")
    print(" - Generate a report about AI")
    print(" - Read AI_report.txt")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_chat())
    await run_chat()


Artifact Demo Started
Try:
 - Generate a report about AI
 - Read AI_report.txt
Type 'exit' to quit.



You:  Hey


Assistant: Hello! How can I help you today?



You:  Generate a report about AI


Assistant: I've generated a report about AI and saved it as 'AI_report.txt' (version 0).



You:  Read report AI_report.txt


Assistant: Here's the content of 'AI_report.txt':

===== REPORT =====
Topic: AI

This is a generated report about AI.




You:  


## **<font color="green">Core Concepts of Artifacts</font>**
Artifacts involves grasping a few key components: the service that manages them, the data structure used to hold them, and how they are identified and versioned.
### **<font color="blue">Artifact Service (`BaseArtifactService`)</font>**
- **Role:** The central component responsible for the actual storage and retrieval logic for artifacts. It defines _how_ and _where_ artifacts are persisted.
- **Interface:** Defined by the abstract base class `BaseArtifactService`.
  - `Save Artifact`: Stores the artifact data and returns its assigned version number.
  - `Load Artifact`: Retrieves a specific version (or the latest) of an artifact.
  - `List Artifact Keys`: Lists the unique filenames of artifacts within a given scope.
  - `Delete Artifact`: Remove a specific version (or the latest) of an artifact.
  - `List Versions`: Lists all available version numbers for a specific artifact filename.
- **Configurable:** You provide and instance(object) of an artifact service (e.g., `InMemoryArtifactService`, `GcsArtifactService`) when initializing the `Runner`. The `Runner` then makes this service available to agents and tools via the `InvocationContext`


In [13]:
# ============================================================
# ADK Artifact Service Full Demo
# Demonstrates:
# - Save Artifact
# - Load Artifact
# - List Artifact Keys
# - List Versions
# - Delete Artifact
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_service_demo"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: SAVE ARTIFACT
# ============================================================

async def save_artifact_tool(filename: str, content: str, tool_context=None) -> str:

    artifact_part = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=artifact_part
    )

    return f"Artifact '{filename}' saved successfully (version {version})."


# ============================================================
# TOOL 2: LOAD ARTIFACT
# ============================================================

async def load_artifact_tool(filename: str, tool_context=None) -> str:

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        return artifact_part.inline_data.data.decode("utf-8")

    return "Artifact exists but contains no readable data."


# ============================================================
# TOOL 3: LIST ARTIFACT KEYS
# ============================================================

async def list_artifacts_tool(tool_context=None) -> str:

    keys = await tool_context.list_artifacts()

    if not keys:
        return "No artifacts found."

    return "Available Artifacts:\n" + "\n".join(keys)


# ============================================================
# TOOL 4: LIST VERSIONS
# ============================================================

async def list_versions_tool(filename: str, tool_context=None) -> str:

    versions = await tool_context.list_artifact_versions(filename=filename)

    if not versions:
        return "No versions found for this artifact."

    return f"Versions for '{filename}': {versions}"


# ============================================================
# TOOL 5: DELETE ARTIFACT
# ============================================================

async def delete_artifact_tool(filename: str, tool_context=None) -> str:

    await tool_context.delete_artifact(filename=filename)

    return f"Artifact '{filename}' deleted successfully."


# ============================================================
# WRAP TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_artifact_tool),
    FunctionTool(func=load_artifact_tool),
    FunctionTool(func=list_artifacts_tool),
    FunctionTool(func=list_versions_tool),
    FunctionTool(func=delete_artifact_tool),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactUserAgent",
    model=MODEL,
    instruction="""
You are an assistant that manages artifacts.

You can:
- Save artifacts
- Load artifacts
- List artifacts
- List versions
- Delete artifacts

Always use tools when performing artifact operations.
""",
    tools=tools
)

# ============================================================
# RUNNER SETUP WITH ARTIFACT SERVICE
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("=" * 60)
    print("Artifact Service Demo Started")
    print("=" * 60)
    print("Examples:")
    print(" - Save file1.txt with content Hello World")
    print(" - Load file1.txt")
    print(" - List artifacts")
    print(" - List versions of file1.txt")
    print(" - Delete file1.txt")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)


# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_demo())
    await run_demo()


Artifact Service Demo Started
Examples:
 - Save file1.txt with content Hello World
 - Load file1.txt
 - List artifacts
 - List versions of file1.txt
 - Delete file1.txt
Type 'exit' to quit.



You:  Save file1.txt with content Hello World


Assistant: OK. I have saved `file1.txt` with content `Hello World`.


You:  Save file2.txt with content Hey Programmer


Assistant: OK. I have saved `file2.txt` with content `Hey Programmer`.


You:  List artifacts


Assistant: OK. I have listed the artifacts. There are `file1.txt` and `file2.txt`.


You:  exit


### **<font color="blue">Artifact Data</font>**
- **Standard Representation:** Artifact content is universally represented using the `google.genai.types.Part` object, the same structure used for parts of LLM messages.
- **Key Attribute(`inline_data`):** For artifacts, the most relevant attribute is `inline_data`, which is a `google.genai.types.Blob` object containing:
  - **`data` (bytes):** A raw binary content of the artifact.
  - **`mime_type` (str):** A standard MIME type string(e.g., `application/pdf`, `image/png`, `audio/mpeg`) describing the nature of the binary data. This is crucial for correct interpretation when loading the artifact.

In [14]:
# ============================================================
# ADK Artifact Data Demo (Binary + MIME Type Example)
# Demonstrates:
# - types.Part
# - types.Blob (inline_data)
# - MIME type usage
# - Save binary artifact
# - Load and inspect MIME type
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types

from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_data_demo"
USER_ID = "user_1"
SESSION_ID = "session_001"

# ============================================================
# TOOL 1: SAVE PDF ARTIFACT (Binary Example)
# ============================================================

async def save_pdf_tool(filename: str, tool_context=None) -> str:
    """
    Creates a mock PDF binary artifact and saves it.
    """

    # Mock PDF binary data
    pdf_bytes = b"%PDF-1.4\n%Mock PDF File for ADK Demo\n1 0 obj\n<< /Type /Catalog >>\nendobj\n"

    pdf_mime_type = "application/pdf"

    # Create Part using inline_data + Blob
    pdf_artifact = types.Part(
        inline_data=types.Blob(
            data=pdf_bytes,
            mime_type=pdf_mime_type
        )
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=pdf_artifact
    )

    return f"PDF artifact '{filename}' saved successfully (version {version})."


# ============================================================
# TOOL 2: LOAD ARTIFACT AND INSPECT METADATA
# ============================================================

async def load_artifact_tool(filename: str, tool_context=None) -> str:
    """
    Loads artifact and displays MIME type and size.
    """

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return "Artifact not found."

    if artifact_part.inline_data:
        blob = artifact_part.inline_data
        size = len(blob.data)
        mime = blob.mime_type

        return (
            f"Artifact Loaded Successfully\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Size: {size} bytes"
        )

    return "Artifact has no inline binary data."


# ============================================================
# TOOL 3: LIST ARTIFACTS
# ============================================================

async def list_artifacts_tool(tool_context=None) -> str:

    keys = await tool_context.list_artifacts()

    if not keys:
        return "No artifacts stored."

    return "Stored Artifacts:\n" + "\n".join(keys)


# ============================================================
# WRAP TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_pdf_tool),
    FunctionTool(func=load_artifact_tool),
    FunctionTool(func=list_artifacts_tool),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="ArtifactDataAgent",
    model=MODEL,
    instruction="""
You manage binary artifacts.

You can:
- Save a PDF artifact
- Load an artifact
- Show MIME type
- List stored artifacts

Always use tools when performing artifact operations.
""",
    tools=tools
)

# ============================================================
# RUNNER
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    print("=" * 60)
    print("Artifact Data (Binary + MIME Type) Demo Started")
    print("=" * 60)
    print("Examples:")
    print(" - Save sample.pdf")
    print(" - Load sample.pdf")
    print(" - List artifacts")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", ""]:
            break

        content = types.Content(
            role="user",
            parts=[types.Part(text=user_input)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)


# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    # asyncio.run(run_demo())
    await run_demo()



Artifact Data (Binary + MIME Type) Demo Started
Examples:
 - Save sample.pdf
 - Load sample.pdf
 - List artifacts
Type 'exit' to quit.



You:  save sample.pdf


Assistant: PDF artifact 'sample.pdf' saved successfully (version 0).


You:  Load sample.pdf


Assistant: Artifact Loaded Successfully
Filename: sample.pdf
MIME Type: application/pdf
Size: 73 bytes


You:  list artifacts


Assistant: Stored Artifacts:
sample.pdf


You:  exit


### **<font color="blue">Filename</font>**
- **Identifier:** A simple string used to name and retrieve an artifact within its specific namespace.
- **Uniqueness:** Filenames must be unique within their scope (either the session or the user namespace).
- **Best Practice:** Use descriptive names, potentially including file extensions(e.g., `"monthly_report.pdf"`, `"user_avatar.jpg"`, although the extension itself doesn't dictate behavior-the `mime_type` does.
### **<font color="blue">Versioning</font>**
- **Automatic Versioning:** The artifact service automatically handles versioning When you call `save_artifact`, the service determines the next available version number (typically starting from 0 and incrementing) for that specific filename and scope.
- **Returned by `save_artifact`:** Then `save_artifact` method returns the integer version number that was assigned to the newly saved artifact.
- **Retrieval:**
  - **`load_artifact(..., version=None)` (default):** Retrieves the _latest_ available version of the artifact.
  - **`load_artifact(..., version=N)`:** Retrieves the specific version `N`.
- **Listing Versions:** The `list_versions` method (on the service, not context) can be used to find all existing version numbers for an artifact.
### **<font color="blue">Namespacing (Session vs. User)</font>**
- **Concept:** Artifacts can be scoped either to a specific `session` or more broadly to a user across all their **sessions** within the applicaiton. This scope is determined by the `filename` format and handled internally by the `ArtifactService`.
- **Default (Session Scope):** If you use a plain filename like `"report.pdf"`, the artifact is associated with the specific `app_name`, `user_id`, and `session_id`. It's only accessible within that exact session context.
- **User Scope (`"user:"` prefix):** If you prefix the finename with `"user:"`, like `"user:profile.png"`, the artifact is associated only with the `app_name` and `user_id`. It can be accessed or updated from _any_ session belogning to that user within the app.

In [16]:
# ============================================================
# ADK Artifact Namespace Demo (Session vs User Scope)
# Fully Corrected Version
# ============================================================

import os
import asyncio
import json

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
import google.genai.types as types
from config import config

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_namespace_demo"
USER_ID = "user_1"

SESSION_ID_1 = "session_A"
SESSION_ID_2 = "session_B"

# ============================================================
# TOOL 1: SAVE SESSION ARTIFACT (Session Scoped)
# ============================================================

async def save_session_artifact(tool_context=None):

    content = "This is a session-specific summary."

    artifact = types.Part.from_bytes(
        data=content.encode("utf-8"),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename="summary.txt",  # session scoped
        artifact=artifact
    )

    return f"Session artifact 'summary.txt' saved (version {version})."


# ============================================================
# TOOL 2: SAVE USER ARTIFACT (User Scoped via prefix)
# ============================================================

async def save_user_settings(tool_context=None):

    settings = {
        "theme": "dark",
        "language": "en",
        "notifications": True
    }

    json_bytes = json.dumps(settings, indent=2).encode("utf-8")

    artifact = types.Part(
        inline_data=types.Blob(
            data=json_bytes,
            mime_type="application/json"
        )
    )

    version = await tool_context.save_artifact(
        filename="user:settings.json",  # user scoped
        artifact=artifact
    )

    return f"User artifact 'user:settings.json' saved (version {version})."


# ============================================================
# TOOL 3: LOAD ARTIFACT
# ============================================================

async def load_artifact(filename: str, tool_context=None):

    artifact_part = await tool_context.load_artifact(filename=filename)

    if not artifact_part:
        return f"Artifact '{filename}' not found."

    if artifact_part.inline_data:
        data = artifact_part.inline_data.data.decode("utf-8")
        mime = artifact_part.inline_data.mime_type

        return (
            f"Artifact Loaded:\n"
            f"Filename: {filename}\n"
            f"MIME Type: {mime}\n"
            f"Content:\n{data}"
        )

    return "Artifact exists but has no inline data."


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=save_session_artifact),
    FunctionTool(func=save_user_settings),
    FunctionTool(func=load_artifact),
]

# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="NamespaceAgent",
    model=MODEL,
    instruction="""
You manage artifacts with namespace scopes.

Session artifact:
- summary.txt

User artifact:
- user:settings.json

Use tools when saving or loading artifacts.
""",
    tools=tools
)

# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # Create sessions (FIXED: keyword arguments required)
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_1
    )

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_2
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(session_id, message_text):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message_text)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content
        )

        async for event in events:
            if event.is_final_response():
                print("Assistant:", event.content.parts[0].text)

    print("\n=== SESSION A: Saving Artifacts ===\n")

    await invoke(SESSION_ID_1, "Save session artifact")
    await invoke(SESSION_ID_1, "Save user settings")

    print("\n=== SESSION B: Testing Visibility ===\n")

    # Should NOT see session artifact
    await invoke(SESSION_ID_2, "Load summary.txt")

    # Should see user artifact
    await invoke(SESSION_ID_2, "Load user:settings.json")


# ============================================================
# EXECUTION
# ============================================================

# If running in Jupyter:
await run_demo()

# If running as normal Python script instead, use:
# if __name__ == "__main__":
#     asyncio.run(run_demo())



=== SESSION A: Saving Artifacts ===

Assistant: Okay, I have saved the session artifact 'summary.txt'.

Assistant: Okay, I have saved the user artifact 'user:settings.json'.


=== SESSION B: Testing Visibility ===

Assistant: I was unable to load the artifact. It appears that the artifact 'summary.txt' does not exist. Did you mean to load a different artifact? Or, perhaps the artifact was not saved.
Assistant: OK. I have loaded the artifact 'user:settings.json'. Here is the content:

```
{
  "theme": "dark",
  "language": "en",
  "notifications": true
}
```



## **<font color="green">Interacting with Artifacts (via Context Objects)</font>**
### **<font color="blue">Prerequisite: Configuring the ArtifactService</font>**
### **<font color="blue">Accessing Methods</font>**
-  Saving Artifacts
-  Loading Artifacts
-  Listing Artifacts Filenames

## **<font color="green">Available Implementations</font>**
### **<font color="blue">InMemoryArtifactService</font>**
- Storage Mechanism
- Key Features
  - Simplicity
  - Speed
  - Ephemeral
- Use Cases
- Installation
### **<font color="blue">GcsArtifactService</font>**
-  Storage Mechanism
-  Object Naming Convention
-  Key Features
   - Persistence
   - Scalability
   - Versioning
   - Permissions Required
-  Use Cases
-  Installation